# Citibike ML
In this example we use the [Citibike dataset](https://ride.citibikenyc.com/system-data). Citibike is a bicycle sharing system in New York City. Everyday users choose from 20,000 bicycles at 1300 stations around New York City.

To ensure customer satisfaction Citibike needs to predict how many bicycles will be needed at each station. Maintenance teams from Citibike will check each station and repair or replace bicycles. Additionally, the team will relocate bicycles between stations based on predicted demand. The business needs to be able to run reports of how many bicycles will be needed at a given station on a given day.

## ML Ops
In this section of the demo, we will utilize Snowpark's Python client-side Dataframe API as well as the Snowpark server-side runtime to create an **ML ops pipeline**.  We will take the functions created by the ML Engineer and create a set of functions that can be easily automated with the company's orchestration tools. 

The ML Engineer must create a pipeline to **automate deployment** of models and batch predictions where the business users can consume them easily from dashboards and analytics tools like Tableau or Power BI.  Predictions will be made for the top 10 busiest stations.  The predictions must be accompanied by an explanation of which features were most impactful for the prediction.  

For this demo flow we will assume that the organization has the following **policies and processes** :   
-**Dev Tools**: The ML engineer can develop in their tool of choice (ie. VS Code, IntelliJ, Pycharm, Eclipse, etc.).  Snowpark Python makes it possible to use any environment where they have a python kernel.  For the sake of a demo we will use Jupyter.  
-**Data Governance**: To preserve customer privacy no data can be stored locally.  The ingest system may store data temporarily but it must be assumed that, in production, the ingest system will not preserve intermediate data products between runs. Snowpark Python allows the user to push-down all operations to Snowflake and bring the code to the data.   
-**Automation**: Although the ML engineer can use any IDE or notebooks for development purposes the final product must be python code at the end of the work stream.  Well-documented, modularized code is necessary for good ML operations and to interface with the company's CI/CD and orchestration tools.  
-**Compliance**: Any ML models must be traceable back to the original data set used for training.  The business needs to be able to easily remove specific user data from training datasets and retrain models. 

Input: Data in `trips` table.  Feature engineering, train, predict functions from data scientist.  
Output: Automatable pipeline of feature engineering, train, predict.

### 1. Load  credentials and connect to Snowflake

In [ ]:
import snowflake.snowpark as snp

from datetime import datetime
import json
import getpass
import uuid

with open('creds.json') as f:
    data = json.load(f)
    connection_parameters = {
      'account': data['account'],
      'user': data['username'],
      'password': data['password'], #getpass.getpass(),
      'role': data['role'],
      'warehouse': data['warehouse']}

session = snp.Session.builder.configs(connection_parameters).create()

### 1. Setup Pipeline

In [ ]:
from citibike_ml.mlops_pipeline import deploy_pred_train_udf
from citibike_ml.mlops_pipeline import materialize_holiday_weather
from citibike_ml.mlops_pipeline import generate_feature_views
from citibike_ml.mlops_pipeline import train_predict_feature_views

In [ ]:
project_db_name = 'CITIBIKEML'
project_schema_name = 'DEMO'
project_db_schema = str(project_db_name)+'.'+str(project_schema_name)

trips_table_name = str(project_db_schema)+'.'+'TRIPS'
holiday_table_name = str(project_db_schema)+'.'+'HOLIDAYS'
precip_table_name = str(project_db_schema)+'.'+'WEATHER'

_ = session.sql('USE DATABASE ' + str(project_db_name)).collect()
_ = session.sql('USE SCHEMA ' + str(project_schema_name)).collect()

The business doesn't actively maintain bicycle stock at EVERY station.  We only need predictions for the `top_n` number of stations.  Initially that is 10 but it might change.

In [ ]:
top_n = 10

We will deploy the model training and inference as a permanent [Python Snowpark User-Defined Function (UDF)](https://docs.snowflake.com/en/LIMITEDACCESS/snowpark-python.html#creating-user-defined-functions-udfs-for-dataframes). This will make the function available to not only our automated training/inference pipeline but also to any users needing the function for manually generated predictions.  
  
As a permanent function we will need a staging area.

In [ ]:
model_stage_name = str(project_db_schema)+'.'+'model_stage'
_ = session.sql('CREATE STAGE IF NOT EXISTS model_stage').collect()

For production we need to be able to reproduce results.  The `trips` table will change as new data is loaded each month so we need a point-in-time snapshot.  Snowflake [Zero-Copy Cloning](https://docs.snowflake.com/en/sql-reference/sql/create-clone.html) allows us to do this with copy-on-write features so we don't have multiple copies of the same data.  We will create a unique ID to identify each training/inference run as well as the features and predictions generated.  We can use [object tagging](https://docs.snowflake.com/en/user-guide/object-tagging.html) to tag each object with the `model_id` as well.

In [ ]:
model_id = str(uuid.uuid1()).replace('-', '_')

clone_table_name = str(project_db_schema)+'.'+'CLONE_'+str(model_id)
_ = session.sql('CREATE OR REPLACE TABLE '+str(clone_table_name)+" CLONE "+str(trips_table_name)).collect()
_ = session.sql('CREATE TAG IF NOT EXISTS model_id_tag').collect()
_ = session.sql("ALTER TABLE "+str(clone_table_name)+" SET TAG model_id_tag = '"+str(model_id)+"'").collect()

feature_view_name = str(project_db_schema)+'.'+'STATION_<station_id>_VIEW_'+str(model_id)

pred_table_name = str(project_db_schema)+'.'+'PREDICTIONS_'+str(model_id)
_ = session.sql('DROP TABLE IF EXISTS '+pred_table_name).collect()

We will start by importing the functions created by the ML Engineer.

The pipeline will be orchestrated by our companies orchestration framework but we will test the steps here.

In [ ]:
%%time 
model_udf_name = deploy_pred_train_udf(session=session, 
                                       function_name='station_train_predict_func', 
                                       model_stage_name=model_stage_name)

In [ ]:
%%time 
holiday_table_name, precip_table_name = materialize_holiday_weather(session, 
                                                                   trips_table_name=trips_table_name, 
                                                                   holiday_table_name=holiday_table_name, 
                                                                   precip_table_name=precip_table_name)

In [ ]:
%%time 
feature_view_names = generate_feature_views(session=session, 
                                            clone_table_name=clone_table_name, 
                                            feature_view_name=feature_view_name, 
                                            holiday_table_name=holiday_table_name, 
                                            precip_table_name=holiday_table_name,
                                            target_column='COUNT', 
                                            top_n=top_n)

In [ ]:
%%time 
pred_table_name = train_predict_feature_views(session=session, 
                                               station_train_pred_udf_name=model_udf_name, 
                                               feature_view_names=feature_view_names, 
                                               pred_table_name=pred_table_name)

In [ ]:
session.table(pred_table_name).select('STATION_ID').distinct().count() #.show()